In [1]:
import pandas as pd
import ast
import re
import datasets
from datasets import load_dataset
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
)
from guidance import models, select
from langchain_core.prompts import PromptTemplate
from operator import itemgetter


import warnings
warnings.filterwarnings("ignore")

/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


In [2]:
# Definisci una funzione di pulizia per rimuovere caratteri non validi
def clean_text(text):
    return re.sub(r"[^\w\s.,!?\-:;()]+", '', text)

# Definisci una funzione di pulizia per rimuovere caratteri non validi
def clean_text_final(text):
    text = re.sub(r'[^\w\s.,!?\'"\-:;()]+', '', text)  # Rimuove i caratteri speciali
    text = re.sub(r"['\"-]", '', text)  # Rimuove apostrofi, virgolette e trattini
    text = text.lower()  # Converte in minuscolo
    return text

In [3]:
# prompt augmentation for the (format of the) synthesis:
prompt_template = PromptTemplate.from_template(
"""You are a multiple-choice question answering assistant.
Choose the most proper option between {options} that best matches with the suggestion. 

Question: {question}
Context: {critique}
Sources: {context}

Assistant:
"""
)
augmentation = {"question": itemgetter("question"),
                "options": itemgetter("options"), 
                "critique": itemgetter("critique"),
                "context": itemgetter("context"), }
synthesis_chain = augmentation | prompt_template 

In [4]:
def create_message_thesis(question, options, context):
    options_str = '", "'.join(options)
    content = f"""

    Now do the same for this question: "{question}", where options: ["{options_str}"]. Assistant:
    """

    user_content = "Answer to the following question: " + question + " providing one of these options as answer: " + str(options) + "Assistant:"

    messages = [
        {"role": "system", "content": """
        You are an helpful AI assistant. You have to provide helpful answers to the user’s questions based on the context: 
        """ + context},
        {"role": "user", "content": user_content}
    ]

    return messages

def extract_answer_thesis(text):
    # Trova l'indice in cui inizia il testo "Why or why not the answer is correct:"
    start_index = text.find("}]")

    
    # Se l'indice è stato trovato, estrai la risposta corretta
    if start_index != -1:
        start_index += len("}]")
        # Estrai il testo dopo "Why or why not the answer is correct:"
        correct_answer_text = text[start_index:].strip()
        return correct_answer_text
    else:
        return "The correct answer could not be found."

def thesisGeneration(query, merged, sources):
    merged = ast.literal_eval(merged)
    augmented_prompt = create_message_thesis(query, merged, sources)
    ans = new_model + str(augmented_prompt) + select(merged)
    return str(ans)

In [5]:
def create_message_antithesis(question, candidate, options, context):
    options_str = '", "'.join(options)
    content = f"""

    Now do the same for this question: "{question}", where options: ["{options_str}"]. Assistant:
    """

    user_content = "Question: " + question + "\n Options: " + str(options) + "\n Candidate answer: " + candidate + "\n Context: " + context + "\n Assistant: \n"

    messages = [
        {"role": "system", "content": """
        You are an helpful AI assistant. You are asked to determine the most correct answer for a given question, provided a set of possible options.
        You also have at disposal a first tentative answer that you are required to check with respect to the question and the relevant context.
        Your goal is to decree which is the most correct answer to the question between the available options.

        Here's an example of how to do it:
        """},
        {"role": "user", "content": """
        Question: What event did Juan Rossell participate in?
        Options: ['1996 summer olympics', 'olympic games', 'sport']
        Candidate answer: 1996 summer olympics
        Context: The 2004 Summer Olympic Games, held in Athens, Greece, marked the return of the games to their birthplace. With a motto of "Welcome Home," the event featured 10,625 athletes from 201 countries competing in 301 medal events across 28 sports. This edition of the Games was significant as it was the first time all countries with a National Olympic Committee participated.\n\nIn the realm of beach volleyball, a sport introduced to the Olympics in 1996, Juan Miguel Rossell Milanes from Cuba emerged as a notable figure. He won the gold medal in the men\'s beach team competition at the 2003 Pan American Games in Santo Domingo, Dominican Republic, partnering Francisco Alvarez. Rossell Milanes also represented Cuba at the 1996 and 2004 Summer Olympics.\n\nThe Pan American Games, a major sporting event in the Americas, also featured beach volleyball as a competitive discipline. The 14th Pan American Games took place in Santo Domingo, Dominican Republic, in 2003. The successful bid for the games was made in the mid-1990s, when the Dominican Republic experienced one of the highest growth rates in Latin America.\n\nThe International Olympic Committee (IOC) voted in 1986 to separate the Summer and Winter Games, which had been held in the same year since 1924, and place them in alternating even-numbered years, beginning in 1994. The 1996 Summer Games were the first to be staged in a different year from the Winter Games. Atlanta, Georgia, United States, hosted the 1996 Summer Games, becoming the fifth American city to host the Olympic Games and the third to hold a Summer Olympic Games.

        Assistant: Let's consider each option and check whether or not is the proper one with respect to the context.'sport' is an extremely generic answer, inappropriate for the question asked. 'olympic games' is not the most proper answer, since it is true that Juan Rossell introduced beach volleyball as an olympic sport, but he participated only at Summer Olympics, both in 1996 and in 2003. Therefore, the correct option is '1996 summer olympics'.

        """
        },
        {"role": "system", "content": "Now do the same for the following question:"},
        {"role": "user", "content": user_content}
    ]

    return messages

def antithesisGeneration(query, merged, candidate, sources):
    merged = ast.literal_eval(merged)
    prompt = create_message_antithesis(query, candidate, merged, sources)
    output = pipe(prompt, **generation_args)
    return output[0]['generated_text']

In [6]:
def create_message_presynthesis(question, candidate, suggestion, options, context):

    user_content = "Question: " + question + "\n Options: " + str(options) + "\n Candidate answer: " + candidate + "\n Suggestion: " + suggestion + "\n Context: " + context + "\n Assistant: \n"

    messages = [
        {"role": "system", "content": """
        You are an helpful AI assistant. You are asked to determine the most correct answer for a given question, provided a set of possible options.
        You also have at disposal a first tentative answer and a suggestion on which is the correct answer.
        Your goal is to decree which is the most correct answer to the question between the available options according to the context.

        Here's a few examples on how to do it:
        """},
        {"role": "user", "content": """
        Question: What event did Juan Rossell participate in?
        Options: ['1996 summer olympics', 'olympic games', 'sport']
        Candidate answer: 1996 summer olympics
        Suggestion: Let's consider each option and check whether or not is the proper one with respect to the context.'sport' is an extremely generic answer, inappropriate for the question asked. 'olympic games' is not the most proper answer, since it is true that Juan Rossell introduced beach volleyball as an olympic sport, but he participated only at Summer Olympics, both in 1996 and in 2003. Therefore, the correct option is '1996 summer olympics'.
        Context: The 2004 Summer Olympic Games, held in Athens, Greece, marked the return of the games to their birthplace. With a motto of "Welcome Home," the event featured 10,625 athletes from 201 countries competing in 301 medal events across 28 sports. This edition of the Games was significant as it was the first time all countries with a National Olympic Committee participated.\n\nIn the realm of beach volleyball, a sport introduced to the Olympics in 1996, Juan Miguel Rossell Milanes from Cuba emerged as a notable figure. He won the gold medal in the men\'s beach team competition at the 2003 Pan American Games in Santo Domingo, Dominican Republic, partnering Francisco Alvarez. Rossell Milanes also represented Cuba at the 1996 and 2004 Summer Olympics.\n\nThe Pan American Games, a major sporting event in the Americas, also featured beach volleyball as a competitive discipline. The 14th Pan American Games took place in Santo Domingo, Dominican Republic, in 2003. The successful bid for the games was made in the mid-1990s, when the Dominican Republic experienced one of the highest growth rates in Latin America.\n\nThe International Olympic Committee (IOC) voted in 1986 to separate the Summer and Winter Games, which had been held in the same year since 1924, and place them in alternating even-numbered years, beginning in 1994. The 1996 Summer Games were the first to be staged in a different year from the Winter Games. Atlanta, Georgia, United States, hosted the 1996 Summer Games, becoming the fifth American city to host the Olympic Games and the third to hold a Summer Olympic Games.
        """
        },
        {"role": "assistant", "content": """
        Assistant: The correct option is '1996 summer olympics', both the candidate answer and the suggestion agree.
        """
        },
        

        
        {"role": "system", "content": "Now do the same for the following question:"},
        {"role": "user", "content": user_content}
    ]

    return messages

def preSynthGeneration(query, candidate_answer, critique, merged, sources):
    prompt = create_message_presynthesis(query, merged, candidate_answer, critique, sources)
    output = pipe(prompt, **generation_args)
    return output[0]['generated_text']

In [7]:

def synthesisGeneration(query, merged, pre_answer, sources):
    merged = ast.literal_eval(merged)
    augmented_prompt = synthesis_chain.invoke({'question': query, 
                                            'options': merged,
                                            'critique': pre_answer,
                                            'context': sources})

    normal_string = clean_text(augmented_prompt.text)
    ans = new_model + normal_string + select(merged)
    return str(ans)

def extract_answer_synthesis(text):
    # Trova l'indice in cui inizia il testo "Why or why not the answer is correct:"
    start_index = text.find("Assistant:\n")

    
    # Se l'indice è stato trovato, estrai la risposta corretta
    if start_index != -1:
        start_index += len("Assistant:\n")
        # Estrai il testo dopo "Why or why not the answer is correct:"
        correct_answer_text = text[start_index:].strip()
        return correct_answer_text
    else:
        return "The correct answer could not be found."

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct", use_fast=False)
new_model = models.Transformers(model, tokenizer, temperature=0.0)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "do_sample": False,
}

In [10]:
df = pd.read_csv('../../../wikihop_dataset/wikihop-merged-summarized.csv')

# select a subset of the queries, just for test:
first_queries = df['query']

# same for correct answers and distractors:
correct_answers = df['answer']
possibilities = df['options']

# and for the sources:
sources = df['sum_supports']

N_rows = 5

In [3]:
possibilities[1]

"['english', 'greek', 'koine greek', 'nahuatl', 'spanish']"

In [4]:
sources[1]

' Christianity is a monotheistic religion based on the life and teachings of Jesus Christ. It is the largest religion in the world, with over 2 billion followers. Christians believe in one God and follow the teachings of Jesus Christ, who they believe is the Son of God and the savior of humanity.\n\nLakewood Church is a nondenominational charismatic Christian megachurch located in Houston, Texas. It is the largest congregation in the United States, with an average attendance of about 52,000 people per week. The church is led by Joel Osteen, who is the son of John Hillery Osteen, the founder and first pastor of Lakewood Church.\n\nMexico is a federal republic in North America, bordered by the United States, Guatemala, Belize, the Caribbean Sea, and the Pacific Ocean. It is the most populous Spanish-speaking country in the world and the second most populous country in Latin America. Mexico is a federation comprising 31 states and a federal district that is also its capital and most popul

In [5]:
first_queries[1]

'What languages did John Osteen speak or write?'

In [13]:
# THESIS
answers = []
for i in range(N_rows):
    answers.append(extract_answer_thesis(thesisGeneration(first_queries[i], possibilities[i], sources[i])))


# ANTITHESIS
ant_answers = []
for i in range(N_rows):
    ant_answers.append(antithesisGeneration(first_queries[i], possibilities[i], answers[i], sources[i]))

# SYNTHESIS
pre_answers = []
for i in range(N_rows):
    pre_answers.append(preSynthGeneration(first_queries[i], possibilities[i], answers[i], ant_answers[i], sources[i]))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [14]:
pre_answers

[" The correct option is '1996 summer olympics'. Both the candidate answer and the suggestion agree. <|end|>",
 " The correct options are 'english' and'spanish'. John Osteen, as an American pastor, likely speaks and writes 'english'. However, since he has connections to Mexico through his church, it is also possible that he speaks and writes'spanish'. Both 'english' and'spanish' are correct answers.  <|end|>",
 " The correct option is 'tetrapoda'. The candidate answer and the suggestion agree.\n\nTetrapoda is the most appropriate answer because it is the taxonomic group that includes reptiles. Proaigialosaurus is a genus of extinct reptiles, and therefore, it belongs to the group Tetrapoda. The other options are either specific orders or families within the reptile group, or they are not related to reptiles at all.\n\nHere's a breakdown of the options:\n\n- 'alligatoridae' and 'gavialidae' are families within the order Crocodilia, which is a subset of reptiles.\n- 'amphibia' and 'lissa

In [15]:
correct_answers[:5]

0    1996 summer olympics
1                 english
2            lepidosauria
3              crocodilia
4               physicist
Name: answer, dtype: object